In [23]:
import pandas as pd #Dataframe, Series
import numpy as np

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
import seaborn as sns

import tensorflow as tf

import graphviz
import pydotplus
import io

from scipy import misc

from sklearn.metrics import accuracy_score

import spotipy
import spotipy.util as util
sp = spotipy.Spotify() 
from spotipy.oauth2 import SpotifyClientCredentials 

# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout


%matplotlib inline

In [24]:
cid ="0cadd882a6ab4ff485c80b8b02aa3b0c" 
secret = "04d0f737e18a4a92abee1da25d70766b"
username = ""

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

scope = 'user-library-read playlist-read-private'
token = util.prompt_for_user_token(username, scope)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [25]:
good_playlist = sp.user_playlist("1287242681", "5OdH7PmotfAO7qDGxKdw3J")
bad_playlist = sp.user_playlist("1287242681", "3ySDAXYGUwRrp8C4ejIm9m")

#Get the good song id's out of the good playlists, need it for the audio features call.
good_tracks = good_playlist["tracks"]
good_songs = good_tracks["items"] 
while good_tracks['next']:
    good_tracks = sp.next(good_tracks)
    for item in good_tracks["items"]:
        good_songs.append(item)
good_ids = [] 
print(len(good_songs))
for i in range(len(good_songs)- 500):
    good_ids.append(good_songs[i]['track']['id'])
    
    #Now do the same thing for the bad playlist
bad_tracks = bad_playlist["tracks"]
bad_songs = bad_tracks["items"] 
while bad_tracks['next']:
    bad_tracks = sp.next(bad_tracks)
    for item in bad_tracks["items"]:
        bad_songs.append(item)
bad_ids = [] 
print(len(bad_songs))
for i in range(len(bad_songs)):
    bad_ids.append(bad_songs[i]['track']['id'])
    
#Here is where we get all of the audio_features for the tracks on the good and bad playlists.
features = []
inSavedTracks = []
for i in range(0,len(good_ids),50):
    audio_features = sp.audio_features(good_ids[i:i+50])
    for track in audio_features:
        features.append(track)
        features[-1]['target'] = 1

for i in range(0,len(bad_ids),50):
    audio_features = sp.audio_features(bad_ids[i:i+50])
    for track in audio_features:
        features.append(track)
        features[-1]['target'] = 0
        
trainingData = pd.DataFrame(features)
trainingData.head()

2276
2226


,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,target,tempo,time_signature,track_href,type,uri,valence
0,0.161,https://api.spotify.com/v1/audio-analysis/7eEl...,0.787,197236,0.484,7eElVgPcxDqmgGrSwVFI74,0.004740,6,0.1350,-10.338,1,0.1960,1,107.925,4,https://api.spotify.com/v1/tracks/7eElVgPcxDqm...,audio_features,spotify:track:7eElVgPcxDqmgGrSwVFI74,0.485
1,0.223,https://api.spotify.com/v1/audio-analysis/02oA...,0.757,189796,0.437,02oAUzv4M0ItuTDy2RT3IT,0.000001,10,0.0921,-5.540,1,0.0625,1,106.953,4,https://api.spotify.com/v1/tracks/02oAUzv4M0It...,audio_features,spotify:track:02oAUzv4M0ItuTDy2RT3IT,0.232
2,0.285,https://api.spotify.com/v1/audio-analysis/5q4B...,0.646,271088,0.376,5q4BpnMrYEFzLO0dYODj6J,0.009510,0,0.0899,-10.054,1,0.0478,1,123.172,4,https://api.spotify.com/v1/tracks/5q4BpnMrYEFz...,audio_features,spotify:track:5q4BpnMrYEFzLO0dYODj6J,0.142
3,0.920,https://api.spotify.com/v1/audio-analysis/6Kkt...,0.575,171728,0.220,6Kkt27YmFyIFrcX3QXFi2o,0.000000,10,0.0962,-11.178,0,0.0992,1,81.601,4,https://api.spotify.com/v1/tracks/6Kkt27YmFyIF...,audio_features,spotify:track:6Kkt27YmFyIFrcX3QXFi2o,0.351
4,0.030,https://api.spotify.com/v1/audio-analysis/1Dp7...,0.470,367391,0.450,1Dp7JGFNjvg8Nk0CtMCcnr,0.009280,1,0.0781,-10.963,0,0.2010,1,149.999,4,https://api.spotify.com/v1/tracks/1Dp7JGFNjvg8...,audio_features,spotify:track:1Dp7JGFNjvg8Nk0CtMCcnr,0.175


In [47]:
print(trainingData[trainingData['target']==0])

      acousticness                                       analysis_url  \
1776       0.24200  https://api.spotify.com/v1/audio-analysis/49FG...   
1777       0.06830  https://api.spotify.com/v1/audio-analysis/7136...   
1778       0.04270  https://api.spotify.com/v1/audio-analysis/2hsf...   
1779       0.67300  https://api.spotify.com/v1/audio-analysis/31nI...   
1780       0.28800  https://api.spotify.com/v1/audio-analysis/5cDf...   
1781       0.44700  https://api.spotify.com/v1/audio-analysis/5YWT...   
1782       0.08250  https://api.spotify.com/v1/audio-analysis/53S4...   
1783       0.03250  https://api.spotify.com/v1/audio-analysis/705t...   
1784       0.02570  https://api.spotify.com/v1/audio-analysis/0ttK...   
1785       0.85000  https://api.spotify.com/v1/audio-analysis/3e3g...   
1786       0.11600  https://api.spotify.com/v1/audio-analysis/7ekj...   
1787       0.41000  https://api.spotify.com/v1/audio-analysis/5H4O...   
1788       0.21500  https://api.spotify.com/v1/audi

In [66]:
#Encoding Categorical Data
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

features = ["danceability", "loudness", "valence", "energy", "instrumentalness", "acousticness", "key", "speechiness","duration_ms"]

y = labelencoder.fit_transform(trainingData["target"])
x = trainingData[features]

In [68]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [69]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu', input_dim = 9))

# Adding the second hidden layer
classifier.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the second hidden layer
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the second hidden layer
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the second hidden layer
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))


# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

In [70]:
# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [71]:
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 300)

Epoch 1/300
3201/3201 [==============================] - 1s 260us/step - loss: 0.5568 - acc: 0.6967
Epoch 2/300
3201/3201 [==============================] - 0s 146us/step - loss: 0.4252 - acc: 0.8147
Epoch 3/300
3201/3201 [==============================] - 1s 160us/step - loss: 0.4121 - acc: 0.8201
Epoch 4/300
3201/3201 [==============================] - 0s 142us/step - loss: 0.4040 - acc: 0.8197
Epoch 5/300
3201/3201 [==============================] - 1s 168us/step - loss: 0.3986 - acc: 0.8216
Epoch 6/300
3201/3201 [==============================] - 1s 182us/step - loss: 0.3954 - acc: 0.8222
Epoch 7/300
3201/3201 [==============================] - 1s 209us/step - loss: 0.3888 - acc: 0.8263
Epoch 8/300
3201/3201 [==============================] - 0s 149us/step - loss: 0.3859 - acc: 0.8307
Epoch 9/300
3201/3201 [==============================] - 1s 165us/step - loss: 0.3789 - acc: 0.8294
Epoch 10/300
3201/3201 [==============================] - 0s 145us/step - loss: 0.3756 - acc: 0.8316

3201/3201 [==============================] - 0s 147us/step - loss: 0.2956 - acc: 0.8766
Epoch 83/300
3201/3201 [==============================] - 1s 158us/step - loss: 0.3109 - acc: 0.8725
Epoch 84/300
3201/3201 [==============================] - 0s 152us/step - loss: 0.2927 - acc: 0.8800
Epoch 85/300
3201/3201 [==============================] - 1s 157us/step - loss: 0.2910 - acc: 0.8772
Epoch 86/300
3201/3201 [==============================] - 0s 146us/step - loss: 0.2940 - acc: 0.8803
Epoch 87/300
3201/3201 [==============================] - 0s 151us/step - loss: 0.2926 - acc: 0.8819
Epoch 88/300
3201/3201 [==============================] - 0s 143us/step - loss: 0.2897 - acc: 0.8803
Epoch 89/300
3201/3201 [==============================] - 1s 165us/step - loss: 0.2849 - acc: 0.8857
Epoch 90/300
3201/3201 [==============================] - 0s 146us/step - loss: 0.2907 - acc: 0.8835
Epoch 91/300
3201/3201 [==============================] - 1s 169us/step - loss: 0.2874 - acc: 0.8819
Epo

3201/3201 [==============================] - 0s 141us/step - loss: 0.2210 - acc: 0.9197
Epoch 163/300
3201/3201 [==============================] - 0s 153us/step - loss: 0.2251 - acc: 0.9203
Epoch 164/300
3201/3201 [==============================] - 0s 140us/step - loss: 0.2148 - acc: 0.9256
Epoch 165/300
3201/3201 [==============================] - 0s 152us/step - loss: 0.2201 - acc: 0.9200
Epoch 166/300
3201/3201 [==============================] - 0s 140us/step - loss: 0.2224 - acc: 0.9206
Epoch 167/300
3201/3201 [==============================] - 0s 154us/step - loss: 0.2174 - acc: 0.9197
Epoch 168/300
3201/3201 [==============================] - 0s 140us/step - loss: 0.2213 - acc: 0.9213
Epoch 169/300
3201/3201 [==============================] - 0s 152us/step - loss: 0.2186 - acc: 0.9210
Epoch 170/300
3201/3201 [==============================] - 0s 141us/step - loss: 0.2189 - acc: 0.9228
Epoch 171/300
3201/3201 [==============================] - 0s 152us/step - loss: 0.2158 - acc: 0

3201/3201 [==============================] - 0s 140us/step - loss: 0.1606 - acc: 0.9450
Epoch 243/300
3201/3201 [==============================] - 1s 167us/step - loss: 0.1763 - acc: 0.9394
Epoch 244/300
3201/3201 [==============================] - 0s 139us/step - loss: 0.1672 - acc: 0.9406
Epoch 245/300
3201/3201 [==============================] - 1s 180us/step - loss: 0.1552 - acc: 0.9500
Epoch 246/300
3201/3201 [==============================] - 0s 142us/step - loss: 0.1547 - acc: 0.9497
Epoch 247/300
3201/3201 [==============================] - 1s 166us/step - loss: 0.1611 - acc: 0.9425
Epoch 248/300
3201/3201 [==============================] - 0s 141us/step - loss: 0.1477 - acc: 0.9497
Epoch 249/300
3201/3201 [==============================] - 1s 159us/step - loss: 0.1468 - acc: 0.9538
Epoch 250/300
3201/3201 [==============================] - 0s 140us/step - loss: 0.1559 - acc: 0.9463
Epoch 251/300
3201/3201 [==============================] - 0s 149us/step - loss: 0.1554 - acc: 0

In [72]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)   
print (cm)

[[367  48]
 [ 85 301]]


In [73]:
#playlistToFindSongsYouLikeIn = sp.user_playlist("spotify", "37i9dQZEVXcFqDuJHHirGk")
playlistToFindSongsYouLikeIn = sp.user_playlist("1287242681", "3ySDAXYGUwRrp8C4ejIm9m")
newPlaylist_tracks = playlistToFindSongsYouLikeIn["tracks"]
newPlaylist_songs = newPlaylist_tracks["items"] 
while newPlaylist_tracks['next']:
    newPlaylist_tracks = sp.next(newPlaylist_tracks)
    for song in newPlaylist_tracks["items"]:
        newPlaylist_songs.append(song)
        
newPlaylist_song_ids = [] 
print(len(newPlaylist_songs))
for i in range(len(newPlaylist_songs)):
    newPlaylist_song_ids.append(newPlaylist_songs[i]['track']['id'])
    
newPlaylist_features = []
j = 0
for i in range(0,len(newPlaylist_song_ids),50):
    audio_features = sp.audio_features(newPlaylist_song_ids[i:i+50])
    for track in audio_features:
        track['song_title'] = newPlaylist_songs[j]['track']['name']
        track['artist'] = newPlaylist_songs[j]['track']['artists'][0]['name']
        j= j + 1
        newPlaylist_features.append(track)
print(len(newPlaylist_features))

playlistToLookAtFeatures = pd.DataFrame(newPlaylist_features)

2226
2226


In [77]:
pred = classifier.predict(playlistToLookAtFeatures[features])

likedSongs = 0
i = 0
for prediction in pred:
    if(prediction == 1):
        print ("Song: " + playlistToLookAtFeatures["song_title"][i] + ", By: "+ playlistToLookAtFeatures["artist"][i])
        #sp.user_playlist_add_tracks("1287242681", "7eIX1zvtpZR3M3rYFVA7DF", [test['id'][i]])
        likedSongs= likedSongs + 1
    i = i +1

Song: 16 Days, By: Whiskeytown
Song: Always A Friend, By: Alejandro Escovedo
Song: If It Takes a Lifetime, By: Jason Isbell
Song: Fruits Of My Labor, By: Lucinda Williams
Song: Blue, By: The Jayhawks
Song: Windfall, By: Son Volt
Song: Magnolia Mountain, By: Ryan Adams & The Cardinals
Song: Harlem River Blues, By: Justin Townes Earle
Song: Copperhead Road, By: Steve Earle
Song: Traveling Alone, By: Tift Merritt
Song: Goddamn Lonely Love, By: Drive-By Truckers
Song: Ruby, By: Dave Rawlings Machine
Song: Aerosol Ball, By: The Felice Brothers
Song: Moonshiner, By: Uncle Tupelo
Song: Bobby Fuller Died for Your Sins, By: Chuck Prophet
Song: Breakfast in Hell, By: Slaid Cleaves
Song: Love Hurts - Remastered Version, By: Gram Parsons
Song: If We Were Vampires, By: Jason Isbell and the 400 Unit
Song: The Love That We Need, By: Hayes Carll
Song: Copper Canteen, By: James McMurtry
Song: Love’s Gonna Live Here, By: Buddy Miller
Song: Star Witness, By: Neko Case
Song: I Can't Complain - Live, By: T

Song: Evil Woman, By: Champion Jack Dupree
Song: I Got My Mojo Working, By: The Paul Butterfield Blues Band
